## feature extraction

brain 폴더 안에 각 환자별로 brainimage가 있고, mask 폴더 안에 각 환자별로 mask가 있다. \
brain image m개와 mask n개를 조합하여 각 brain_image-mask쌍에 대해 feature extraction하기 => itertools의 product이용

feature를 extract하여 각 환자가 row에 각 feature가 column에 오도록 csv파일로 뽑아내기

In [1]:
!pip install SimpleITK
!pip install pyradiomics


In [1]:
from glob import glob
import SimpleITK as sitk
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from shutil import move
from itertools import product
import pandas as pd

In [2]:
#PATH = '/media/sc-mlsg/PYW98854602/20210507_GBM_autoseg_preprocessed/'
PATH = '/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/'
in_path = PATH + '04_Stripped_znormalized/'
outpath = PATH
maskDir = PATH + '00_Tumor_masks_resampled/'

In [3]:
for (path, dirs, files) in os.walk(in_path):
    print(path, dirs, files)

/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/ ['8355900', '8704204', '8445407', '5329806', '5165936', '4315438', '10088264', '5764412', '5584298', '7793380', '8274048', '5297962', '7388494', '7940665', '5870222', '6475512', '3016835', '8091567', '8462147', '5545997', '3332002', '5482785', '8573949', '4352863', '5232890', '8854505', '8032803', '3802982', '3819641', '5840185', '8167279', '7618332', '24237', '7837811', '5879269', '5727339', '7547961', '10020902', '8796872', '8688717', '7915638', '8389249', '8188316', '8293084', '5185528', '5148241', '4294621', '5782342', '7648996', '8192913', '7484329', '5247634', '8166115', '890950', '5131557', '7409183', '8573185', '8545834', '10176016', '10162022', '7427402', '7460701', '5162636', '5379798', '8516487', '2752458', '8469528', '5840592'] []
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/8355900 [] ['8355900_T2_registered_znormed.nii.gz', '8355900_T1C_registered_znormed.nii.gz']
/ho

In [4]:
from __future__ import print_function
from radiomics import featureextractor
import radiomics
import logging
radiomics.logger.setLevel(logging.ERROR)
import SimpleITK as sitk
from radiomics import firstorder, glcm, shape, glrlm, glszm, ngtdm, gldm
import numpy as np


def Numpy2Itk(array):
    '''
    :param array: numpy array format
    :return: simple itk image type format
    '''
    return sitk.GetImageFromArray(array)

def feature_extract(image_origin, image_mask, features = ['firstorder', 'glcm', 'glszm', 'glrlm', 'ngtdm', 'shape'], binWidth=32, binCount=None):
    '''
    :param image_origin: image_array (numpy array)
    :param image_mask: mask_array (numpy array)
    :return: whole features, featureVector
    '''
    image = Numpy2Itk(image_origin)
    mask = Numpy2Itk(image_mask)
    
    settings = {}

    if binWidth:
        settings['binWidth'] = binWidth
    if binCount:
        settings['binCount'] = binCount
    settings['resampledPixelSpacing'] = (1,1,1)
    settings['interpolator'] = 'sitkBSpline'
    settings['verbose'] = True
    
    extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
    extractor.settings['enableCExtensions'] = True
    
    for feature in features:
        extractor.enableFeatureClassByName(feature.lower())
        
    featureVector = extractor.execute(image, mask)
    
    cols = []; feats = []
    for feature in features:
        for featureName in sorted(featureVector.keys()):
            if feature in featureName:
                cols.append(featureName)
                feats.append(featureVector[featureName])
    return feats, cols

In [7]:
# check sequence and pt
for (path, dirs, files) in os.walk(in_path):
    if not files:
        continue
    pt = path.split(os.path.sep)[-1]
    img_path = sorted(glob(os.path.join(path, '*')))[0] 
    seq = os.path.basename(img_path).split("_")[1]
    print(seq, pt)

T1C 8355900
T1C 8704204
T1C 8445407
T1C 5329806
T1C 5165936
T1C 4315438
T1C 10088264
T1C 5764412
T1C 5584298
T1C 7793380
T1C 8274048
T1C 5297962
T1C 7388494
T1C 7940665
T1C 5870222
T1C 6475512
T1C 3016835
T1C 8091567
T1C 8462147
T1C 5545997
T1C 3332002
T1C 5482785
T1C 8573949
T1C 4352863
T1C 5232890
T1C 8854505
T1C 8032803
T1C 3802982
T1C 3819641
T1C 5840185
T1C 8167279
T1C 7618332
T1C 24237
T1C 7837811
T1C 5879269
T1C 5727339
T1C 7547961
T1C 10020902
T1C 8796872
T1C 8688717
T1C 7915638
T1C 8389249
T1C 8188316
T1C 8293084
T1C 5185528
T1C 5148241
T1C 4294621
T1C 5782342
T1C 7648996
T1C 8192913
T1C 7484329
T1C 5247634
T1C 8166115
T1C 890950
T1C 5131557
T1C 7409183
T1C 8573185
T1C 8545834
T1C 10176016
T1C 10162022
T1C 7427402
T1C 7460701
T1C 5162636
T1C 5379798
T1C 8516487
T1C 2752458
T1C 8469528
T1C 5840592


In [9]:
error_dict = {}
for size in [8, 32, 128]:
    feat_df_width = pd.DataFrame()
    feat_df_count = pd.DataFrame()
    total = 0
    for (path, dirs, files) in os.walk(in_path):
        if not files:
            continue
        total += 1
        pt = path.split(os.path.sep)[-1]
        print(total, pt)
        brain_paths = sorted(glob(os.path.join(path, '*')))
#        mask_paths = glob(os.path.join(maskDir, pt, '*ce*')) + glob(os.path.join(maskDir, pt, '*peri*'))
        mask_paths = sorted(glob(os.path.join(maskDir, pt, '*')))
        if not mask_paths:
            print("tumor mask not exists")
            continue
        bw_features_lst, bw_cols_lst, bc_features_lst, bc_cols_lst = [], [], [], []
        
        for (img_path, mask_path) in product(brain_paths, mask_paths):
            try:
                seq = os.path.basename(img_path).split("_")[1]
    #                if seq in ["F", "T1", "T2F"]:
    #                    continue
                print(img_path, mask_path)
    #            mask_type = os.path.basename(mask_path).split("_")[0]

                brain_img, mask = [sitk.ReadImage(path) for path in [img_path, mask_path]]
                brain_arr, mask_arr = [sitk.GetArrayFromImage(img) for img in [brain_img, mask]]

                # Scale
#                brain_arr = brain_arr * 100

                # Change binWidth
                bw_features, bw_cols = feature_extract(brain_arr, mask_arr, binWidth=size)
                bw_cols = [col.replace('original', '{}'.format(seq)) for col in bw_cols]
#                bw_cols = [col.replace('original', '{}_image_{}_mask'.format(seq, mask_type)) for col in bw_cols]

                # Change binCount            
                bc_features, bc_cols = feature_extract(brain_arr, mask_arr, binWidth=None, binCount=size)
                bc_cols = [col.replace('original', '{}'.format(seq)) for col in bc_cols]
#                bc_cols = [col.replace('original', '{}_image_{}_mask'.format(seq, mask_type)) for col in bc_cols]

                bw_features_lst += bw_features
                bw_cols_lst += bw_cols
                bc_features_lst += bc_features
                bc_cols_lst += bc_cols

            except Exception as e:
                print("error", e, path)
                error_dict[path] = e
                
        feat_df_width = pd.concat([feat_df_width, pd.DataFrame(np.array(bw_features_lst),index=bw_cols_lst, columns=[pt]).T])
        feat_df_count = pd.concat([feat_df_count, pd.DataFrame(np.array(bc_features_lst), index=bc_cols_lst, columns=[pt]).T])

    feat_df_width.to_csv(os.path.join(outpath, '{}_bin_width_{}.csv'.format('PNS', size)))
    feat_df_count.to_csv(os.path.join(outpath, '{}_bin_count_{}.csv'.format('PNS', size)))
    print("Finished %d size" % size)

1 8355900
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/8355900/8355900_T1C_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/8355900/8355900_mask.nii.gz
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/8355900/8355900_T2_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/8355900/8355900_mask.nii.gz
2 8704204
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/8704204/8704204_T1C_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/8704204/8704204_mask.nii.gz
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/8704204/8704204_T2_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/8704204/8704204_mask.nii.gz
3 8445407
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormaliz

19 8462147
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/8462147/8462147_T1C_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/8462147/8462147_mask.nii.gz
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/8462147/8462147_T2_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/8462147/8462147_mask.nii.gz
20 5545997
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/5545997/5545997_T1C_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/5545997/5545997_mask.nii.gz
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/5545997/5545997_T2_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/5545997/5545997_mask.nii.gz
21 3332002
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znorma

37 7547961
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/7547961/7547961_T1C_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/7547961/7547961_mask.nii.gz
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/7547961/7547961_T2_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/7547961/7547961_mask.nii.gz
38 10020902
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/10020902/10020902_T1C_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/10020902/10020902_mask.nii.gz
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/10020902/10020902_T2_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/10020902/10020902_mask.nii.gz
39 8796872
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripp

55 5131557
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/5131557/5131557_T1C_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/5131557/5131557_mask.nii.gz
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/5131557/5131557_T2_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/5131557/5131557_mask.nii.gz
56 7409183
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/7409183/7409183_T1C_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/7409183/7409183_mask.nii.gz
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/7409183/7409183_T2_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/7409183/7409183_mask.nii.gz
57 8573185
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znorma

5 5165936
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/5165936/5165936_T1C_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/5165936/5165936_mask.nii.gz
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/5165936/5165936_T2_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/5165936/5165936_mask.nii.gz
6 4315438
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/4315438/4315438_T1C_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/4315438/4315438_mask.nii.gz
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/4315438/4315438_T2_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/4315438/4315438_mask.nii.gz
7 10088264
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormali

...

59 10176016
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/10176016/10176016_T1C_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/10176016/10176016_mask.nii.gz
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/10176016/10176016_T2_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/10176016/10176016_mask.nii.gz
60 10162022
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/10162022/10162022_T1C_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/10162022/10162022_mask.nii.gz
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/10162022/10162022_T2_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/10162022/10162022_mask.nii.gz
61 7427402
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/

9 5584298
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/5584298/5584298_T1C_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/5584298/5584298_mask.nii.gz
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/5584298/5584298_T2_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/5584298/5584298_mask.nii.gz
10 7793380
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/7793380/7793380_T1C_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/7793380/7793380_mask.nii.gz
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/7793380/7793380_T2_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/7793380/7793380_mask.nii.gz
11 8274048
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormal

27 8032803
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/8032803/8032803_T1C_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/8032803/8032803_mask.nii.gz
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/8032803/8032803_T2_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/8032803/8032803_mask.nii.gz
28 3802982
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/3802982/3802982_T1C_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/3802982/3802982_mask.nii.gz
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/3802982/3802982_T2_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/3802982/3802982_mask.nii.gz
29 3819641
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znorma

45 5185528
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/5185528/5185528_T1C_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/5185528/5185528_mask.nii.gz
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/5185528/5185528_T2_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/5185528/5185528_mask.nii.gz
46 5148241
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/5148241/5148241_T1C_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/5148241/5148241_mask.nii.gz
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/5148241/5148241_T2_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/5148241/5148241_mask.nii.gz
47 4294621
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znorma

63 5162636
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/5162636/5162636_T1C_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/5162636/5162636_mask.nii.gz
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/5162636/5162636_T2_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/5162636/5162636_mask.nii.gz
64 5379798
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/5379798/5379798_T1C_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/5379798/5379798_mask.nii.gz
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znormalized/5379798/5379798_T2_registered_znormed.nii.gz /home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/00_Tumor_masks_resampled/5379798/5379798_mask.nii.gz
65 8516487
/home/sc-mlsg/Downloads/PNS_nii_final_preprocessed/04_Stripped_znorma

In [ ]:
error_dict

In [30]:
feat_df_width.to_csv(os.path.join(outpath, '{}_bin_width_{}.csv'.format('GBM', size)))
feat_df_count.to_csv(os.path.join(outpath, '{}_bin_count_{}.csv'.format('GBM', size)))


In [31]:
outpath

'/media/sc-mlsg/PYW98854602/20210507_GBM_Preprocessed/'